In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds # to use tensorflow prebuilt datasets
import tensorflow_hub as hub # Contains presaved models for transfer learning

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def clean_text(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "<br />", " ")
    text = tf.strings.regex_replace(text, r"[^a-zA-Z0-9' ]", " ")
    return text

In [ ]:
hub_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# hub_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

class FeatureExtractor(tf.keras.layers.Layer):
  def __init__(self):
    super(FeatureExtractor, self).__init__()
    self.hub_layer  = hub.KerasLayer(hub_url, input_shape=[], dtype=tf.string, trainable=True)
  def call(self, input_tensor):
    return self.hub_layer(input_tensor)

In [ ]:
input_layer = Input(shape=(), dtype=tf.string)

x = FeatureExtractor()(input_layer) 
x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x= Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid', dtype='float32')(x)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 64

model.fit(
  train_data.shuffle(25000).batch(batch_size),
  epochs=20,
  validation_data = validation_data.batch(batch_size),
  verbose = 1,
  callbacks=[callback]
)